In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('data/Consumo_cerveja.csv', 
                 decimal=',', 
                 thousands='.', 
                 header=0, 
                 names=['date','median_temp','min_temp','max_temp','precip','weekend','consumption'], 
                 parse_dates=['date'], 
                 nrows=365)

Let's finish up by talking about a number of useful functions for analysis before we put it all together!

# Mapping
Often we have some data that we want to replace with a better representation. For example,
in our beer drinking data, I would assume that the seasons are important, but since winter actually overlaps 2 years (winter is December, January, February) there's no easy numeric way to divide the year up.

Enter mapping - map is a great convenience function that lets you apply a function or dictionary on every row of data. To create our season mapping, let's set up a dictionary mapping label to number of month:

In [2]:
season = {
    "winter": [12, 1, 2],
    "spring": [3, 4, 5],
    "summer": [6, 7, 8],
    "autumn": [9, 10,11]
}

We then invert the dictionary, so we have a nice representation mapping month number to season

In [3]:
season_map = {i: k
              for k, v in season.items()
              for i in v
             }
season_map

{12: 'winter',
 1: 'winter',
 2: 'winter',
 3: 'spring',
 4: 'spring',
 5: 'spring',
 6: 'summer',
 7: 'summer',
 8: 'summer',
 9: 'autumn',
 10: 'autumn',
 11: 'autumn'}

Something we haven't mentioned yet, but converting date to a datetime type gives us access to the special .dt namespace which has datetime specific functionality. In this example, we use the .month property to get the month of a given date

In [4]:
df.date.dt.month

0       1
1       1
2       1
3       1
4       1
5       1
6       1
7       1
8       1
9       1
10      1
11      1
12      1
13      1
14      1
15      1
16      1
17      1
18      1
19      1
20      1
21      1
22      1
23      1
24      1
25      1
26      1
27      1
28      1
29      1
       ..
335    12
336    12
337    12
338    12
339    12
340    12
341    12
342    12
343    12
344    12
345    12
346    12
347    12
348    12
349    12
350    12
351    12
352    12
353    12
354    12
355    12
356    12
357    12
358    12
359    12
360    12
361    12
362    12
363    12
364    12
Name: date, Length: 365, dtype: int64

Now we are ready to map our season dictionary onto our months

In [5]:
df['season'] = df.date.dt.month.map(season_map)

Let's get some random samples to check that it worked as expected

![Fun Fact](images/fun_fact.resized.jpeg) Use `.sample` instead of `.head` - you'll catch dataerrors from the middle of your dataset!

In [6]:
df.sample(10)

,date,median_temp,min_temp,max_temp,precip,weekend,consumption,season
267,2015-09-25,23.24,19.2,30.3,0.0,0,29411,autumn
45,2015-02-15,22.90,20.0,27.6,17.4,1,24968,winter
119,2015-04-30,16.22,14.7,18.7,0.0,0,20452,spring
264,2015-09-22,25.70,18.9,33.3,0.0,0,26362,autumn
286,2015-10-14,23.62,18.2,28.5,0.0,0,26793,autumn
23,2015-01-24,23.12,19.0,29.4,13.0,1,28348,winter
214,2015-08-03,21.16,14.9,26.9,0.0,0,25867,summer
8,2015-01-09,28.20,21.9,34.0,4.4,0,24886,winter
76,2015-03-18,21.24,19.7,24.1,0.0,0,20167,spring
72,2015-03-14,22.88,19.9,27.9,12.2,1,28844,spring


We can also use the `.value_counts` method to check that everything is as expected

In [7]:
df.season.value_counts()

summer    92
spring    92
autumn    91
winter    90
Name: season, dtype: int64

# Binning

Another very common operation is to want to assign data to a bin. 
For example, we might want to turn a regression problem of predicting consumption into a classification problem of low vs high consumption.

Let's arbitrarily choose 25,000 as our cutoff for high consumption.
Since pandas is based on Numpy, we can often use numpy functions when it suits us.

There is nothing in pandas that does quite what np.where does, so I use it all the time for this type of operation

In [8]:
df['high_consumption'] = np.where(df.consumption < 25000, 0, 1)
df.sample(10)

,date,median_temp,min_temp,max_temp,precip,weekend,consumption,season,high_consumption
75,2015-03-17,21.92,18.6,25.6,24.2,0,26275,spring,1
311,2015-11-08,20.24,17.4,23.9,0.6,1,28610,autumn,1
295,2015-10-23,20.50,17.6,27.3,13.4,0,26594,autumn,1
9,2015-01-10,26.76,22.1,34.2,0.0,1,37937,winter,1
20,2015-01-21,25.32,22.7,30.9,0.0,0,29130,winter,1
6,2015-01-07,24.00,19.5,33.7,0.0,0,29732,winter,1
209,2015-07-29,16.20,11.0,23.8,0.0,0,22522,summer,0
280,2015-10-08,26.56,19.3,34.0,0.0,0,33182,autumn,1
12,2015-01-13,25.52,21.2,34.8,0.8,0,26990,winter,1
4,2015-01-05,23.82,21.0,28.3,0.0,0,28900,winter,1


In [9]:
df.high_consumption.value_counts()

0    188
1    177
Name: high_consumption, dtype: int64

Often our usecase is a bit more complicated than a simple higher or lower. `pd.cut` gives us a lot more flexibility in setting our cutoff points. Let's let pandas figure out the best place to split the data, by specifying that we want 2 bins -this will find the point that divides our data into equal width bins

In [10]:
pd.cut(df.consumption, bins=2).value_counts()

(14319.406, 26140.0]    217
(26140.0, 37937.0]      148
Name: consumption, dtype: int64

While descriptive, that's not very pretty nor easy to select - `pd.cut` also supports passing a list of labels, so let's do that.

In [11]:
pd.cut(df.consumption, bins=2, labels=['low', 'high']).value_counts()

low     217
high    148
Name: consumption, dtype: int64

Same result, but with nice labels for ease of indexing

Of course we can also pass our own bins if we have irregular intervals

In [12]:
pd.cut(df.consumption, bins=[0, 25000, 99999], labels=['low', 'high'])

0      high
1      high
2      high
3      high
4      high
5      high
6      high
7      high
8       low
9      high
10     high
11     high
12     high
13     high
14     high
15     high
16     high
17     high
18     high
19     high
20     high
21     high
22      low
23     high
24     high
25      low
26     high
27      low
28      low
29     high
       ... 
335    high
336    high
337    high
338    high
339    high
340     low
341    high
342    high
343     low
344    high
345    high
346    high
347    high
348    high
349     low
350     low
351    high
352    high
353    high
354     low
355    high
356    high
357    high
358    high
359     low
360    high
361    high
362     low
363     low
364     low
Name: consumption, Length: 365, dtype: category
Categories (2, object): [low < high]

Sometimes you want your bins to be based on quantiles instead of arbitrary intervals - `pd.qcut` makes that easy. 

In [13]:
df['consumption_group'] = pd.qcut(df.consumption, q=3, labels=['low', 'medium', 'high'])

We can of course specify our quantiles explicitly

In [82]:
quartiles = pd.qcut(df.consumption, q=[0, 0.25, 0.75, 1], labels=['low', 'medium', 'high'])
quartiles

date
2015-01-01    medium
2015-01-02      high
2015-01-03      high
2015-01-04      high
2015-01-05      high
2015-01-06    medium
2015-01-07      high
2015-01-08    medium
2015-01-09    medium
2015-01-10      high
2015-01-11      high
2015-01-12    medium
2015-01-13    medium
2015-01-14      high
2015-01-15    medium
2015-01-16      high
2015-01-17      high
2015-01-18      high
2015-01-19      high
2015-01-20      high
2015-01-21      high
2015-01-22    medium
2015-01-23       low
2015-01-24    medium
2015-01-25      high
2015-01-26       low
2015-01-27      high
2015-01-28    medium
2015-01-29    medium
2015-01-30    medium
               ...  
2015-12-02      high
2015-12-03    medium
2015-12-04      high
2015-12-05      high
2015-12-06      high
2015-12-07    medium
2015-12-08    medium
2015-12-09    medium
2015-12-10    medium
2015-12-11      high
2015-12-12      high
2015-12-13      high
2015-12-14    medium
2015-12-15    medium
2015-12-16       low
2015-12-17    medium
2015-12-

# Categoricals

If you looked closely, you might have noticed that the dtype of the Series we got back from `.cut` is a Categorical. A Categorical is a new datatype introduced by Pandas, and so there is no Numpy equivalent.

Categoricals are a neat way of expressing data that represents a category with multiple levels. In this example, we have 3:
- low
- medium
- high

They are also in order from low to high.

The primary benefit from categoricals is saving space - Strings are expensive to store, as we have to store a python object inside each array - very inefficient. Categoricals store codes instead - integers that map to a string label. Just like with dates, where we can access special methods using the `.dt` namespace, we can use `.cat` to use categorical specific methods

In [84]:
quartiles.cat.categories

Index(['low', 'medium', 'high'], dtype='object')

In [83]:
quartiles.cat.codes

date
2015-01-01    1
2015-01-02    2
2015-01-03    2
2015-01-04    2
2015-01-05    2
2015-01-06    1
2015-01-07    2
2015-01-08    1
2015-01-09    1
2015-01-10    2
2015-01-11    2
2015-01-12    1
2015-01-13    1
2015-01-14    2
2015-01-15    1
2015-01-16    2
2015-01-17    2
2015-01-18    2
2015-01-19    2
2015-01-20    2
2015-01-21    2
2015-01-22    1
2015-01-23    0
2015-01-24    1
2015-01-25    2
2015-01-26    0
2015-01-27    2
2015-01-28    1
2015-01-29    1
2015-01-30    1
             ..
2015-12-02    2
2015-12-03    1
2015-12-04    2
2015-12-05    2
2015-12-06    2
2015-12-07    1
2015-12-08    1
2015-12-09    1
2015-12-10    1
2015-12-11    2
2015-12-12    2
2015-12-13    2
2015-12-14    1
2015-12-15    1
2015-12-16    0
2015-12-17    1
2015-12-18    1
2015-12-19    2
2015-12-20    2
2015-12-21    1
2015-12-22    1
2015-12-23    1
2015-12-24    2
2015-12-25    1
2015-12-26    0
2015-12-27    2
2015-12-28    1
2015-12-29    1
2015-12-30    0
2015-12-31    1
Length: 365, dtype:

Now we are storing 365 int8s and 3 strings instead of 365 strings - much nicer!

We can also use categories to indicate data that should be there, but is not - this is great for machine learning production, where you have to encode a row of data into the correct number of categories

In [89]:
example = quartiles.cat.add_categories('example')

In [92]:
example.value_counts()

medium     182
low         92
high        91
example      0
Name: consumption, dtype: int64

# Get Dummies

One common task in datascience, is to one-hot encode categorical columns. As this is also known as "creating dummy variables" pandas has a built in solution for that - `.get_dummies`. It takes your dataframe and one-hot-encodes all categorical columns it finds (usually string columns, but also categoricals). In combination with the categorical dtype from before, it's easy to encode all possible categories, even though they are not present in data

In [94]:
# There are no values labelled 'example' in the data
pd.get_dummies(example)

,low,medium,high,example
date,,,,
2015-01-01,0,1,0,0
2015-01-02,0,0,1,0
2015-01-03,0,0,1,0
2015-01-04,0,0,1,0
2015-01-05,0,0,1,0
2015-01-06,0,1,0,0
2015-01-07,0,0,1,0
2015-01-08,0,1,0,0
2015-01-09,0,1,0,0


You can also pass your full DataFrame to `.get_dummies` - it will generally do the right thing!

In [95]:
pd.get_dummies(df)

,median_temp,min_temp,max_temp,precip,weekend,consumption,high_consumption,season_autumn,season_spring,season_summer,season_winter,consumption_group_low,consumption_group_medium,consumption_group_high
date,,,,,,,,,,,,,,
2015-01-01,27.30,23.9,32.5,0.0,0,25461,1,0,0,0,1,0,1,0
2015-01-02,27.02,24.5,33.5,0.0,0,28972,1,0,0,0,1,0,0,1
2015-01-03,24.82,22.4,29.9,0.0,1,30814,1,0,0,0,1,0,0,1
2015-01-04,23.98,21.5,28.6,1.2,1,29799,1,0,0,0,1,0,0,1
2015-01-05,23.82,21.0,28.3,0.0,0,28900,1,0,0,0,1,0,0,1
2015-01-06,23.78,20.1,30.5,12.2,0,28218,1,0,0,0,1,0,0,1
2015-01-07,24.00,19.5,33.7,0.0,0,29732,1,0,0,0,1,0,0,1
2015-01-08,24.90,19.5,32.8,48.6,0,28397,1,0,0,0,1,0,0,1
2015-01-09,28.20,21.9,34.0,4.4,0,24886,0,0,0,0,1,0,1,0


You can also specify which columns to encode, as well as passing the `drop_first` parameter if you're trying to avoid multicollinearity. 

Note that the `high_consumption` column is an int dtype - all columns passed to `columns` will be one-hot encoded, regardless of datatype.

In [97]:
pd.get_dummies(df, drop_first=True, columns=['season', 'high_consumption'])

,median_temp,min_temp,max_temp,precip,weekend,consumption,consumption_group,season_spring,season_summer,season_winter,high_consumption_1
date,,,,,,,,,,,
2015-01-01,27.30,23.9,32.5,0.0,0,25461,medium,0,0,1,1
2015-01-02,27.02,24.5,33.5,0.0,0,28972,high,0,0,1,1
2015-01-03,24.82,22.4,29.9,0.0,1,30814,high,0,0,1,1
2015-01-04,23.98,21.5,28.6,1.2,1,29799,high,0,0,1,1
2015-01-05,23.82,21.0,28.3,0.0,0,28900,high,0,0,1,1
2015-01-06,23.78,20.1,30.5,12.2,0,28218,high,0,0,1,1
2015-01-07,24.00,19.5,33.7,0.0,0,29732,high,0,0,1,1
2015-01-08,24.90,19.5,32.8,48.6,0,28397,high,0,0,1,1
2015-01-09,28.20,21.9,34.0,4.4,0,24886,medium,0,0,1,0


`.get_dummies` is a great convenience function that is very handy for modelling work!

# Shifting and Diffing

Sometimes, you want to compare running differences - what's the change between days or months? Pandas provides utility methods to do that in various forms

In [16]:
# Subtract the previous value
df.consumption.diff()

0          NaN
1       3511.0
2       1842.0
3      -1015.0
4       -899.0
5       -682.0
6       1514.0
7      -1335.0
8      -3511.0
9      13051.0
10     -1683.0
11    -10511.0
12      1247.0
13      4835.0
14     -6101.0
15      4214.0
16      7752.0
17     -7166.0
18     -1259.0
19      5862.0
20     -5997.0
21     -3335.0
22     -4011.0
23      6564.0
24      2740.0
25     -9568.0
26      8452.0
27     -7369.0
28        93.0
29      4149.0
        ...   
335     5942.0
336    -2066.0
337     1108.0
338     2938.0
339      329.0
340    -9405.0
341     4338.0
342     -576.0
343    -4204.0
344     7807.0
345    -1161.0
346     -391.0
347    -1057.0
348      486.0
349    -7555.0
350     3275.0
351     2705.0
352     5494.0
353    -2409.0
354    -5293.0
355     1994.0
356     -360.0
357     5104.0
358    -5264.0
359    -4353.0
360    10352.0
361    -6212.0
362    -3786.0
363    -1842.0
364     1979.0
Name: consumption, Length: 365, dtype: float64

In [17]:
# Subtract the value from 30 days before
df.consumption.diff(periods=30).dropna()

30      1569.0
31      3085.0
32     -6717.0
33      1856.0
34     -4162.0
35     -8268.0
36     -6911.0
37       496.0
38      5040.0
39    -13875.0
40    -15117.0
41      1062.0
42      -601.0
43     -7606.0
44      4507.0
45     -4970.0
46    -12347.0
47    -13125.0
48     -7873.0
49    -12205.0
50     -4563.0
51      5148.0
52      9041.0
53     -2656.0
54     -4129.0
55      3846.0
56     -7188.0
57      3638.0
58      3771.0
59       630.0
        ...   
335     9823.0
336     5664.0
337     8034.0
338     9317.0
339    12205.0
340     -955.0
341     -897.0
342    -1319.0
343    -5031.0
344     1171.0
345      312.0
346      541.0
347     1295.0
348     -769.0
349    -3547.0
350    -2627.0
351     3428.0
352     9576.0
353     9795.0
354    -5558.0
355    -5105.0
356     4806.0
357     9883.0
358     1189.0
359    -3330.0
360     3328.0
361    -8287.0
362    -8308.0
363      229.0
364    -2083.0
Name: consumption, Length: 335, dtype: float64

In [18]:
# Get the percentage change compared to 7 days ago
df.consumption.pct_change(periods=7)

0           NaN
1           NaN
2           NaN
3           NaN
4           NaN
5           NaN
6           NaN
7      0.115314
8     -0.141033
9      0.231161
10     0.216618
11    -0.109239
12    -0.043518
13     0.070396
14    -0.094130
15     0.203006
16    -0.006511
17    -0.158052
18     0.136814
19     0.301482
20    -0.084682
21     0.002760
22    -0.272363
23    -0.247864
24     0.018477
25    -0.264651
26    -0.146753
27    -0.224065
28    -0.120140
29     0.232326
         ...   
335    0.213066
336    0.123393
337    0.018427
338   -0.056163
339    0.070647
340    0.155005
341    0.129806
342   -0.109416
343   -0.192642
344    0.041575
345   -0.088503
346   -0.109579
347    0.203465
348    0.032620
349   -0.223864
350    0.061222
351   -0.120299
352    0.099970
353    0.032171
354   -0.117202
355   -0.062515
356    0.256671
357    0.297284
358   -0.027143
359   -0.325209
360    0.072360
361    0.050777
362   -0.168443
363   -0.226727
364   -0.289054
Name: consumption, Lengt

These are convenience methods built around `.shift` - `.shift` lets you easily compare a row with another row

In [19]:
# Shift all columns one step
df.shift(1).head()

,date,median_temp,min_temp,max_temp,precip,weekend,consumption,season,high_consumption,consumption_group
0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-01-01,27.30,23.9,32.5,0.0,0.0,25461.0,winter,1.0,medium
2,2015-01-02,27.02,24.5,33.5,0.0,0.0,28972.0,winter,1.0,high
3,2015-01-03,24.82,22.4,29.9,0.0,1.0,30814.0,winter,1.0,high
4,2015-01-04,23.98,21.5,28.6,1.2,1.0,29799.0,winter,1.0,high


Notice how 25,461 was at index 0 previously and is now at index 1

In [20]:
df.consumption.head()

0    25461
1    28972
2    30814
3    29799
4    28900
Name: consumption, dtype: int64

This can be a great way to create a lagged feature set for time series modelling. If you want to predict next days consumption, I can simply create a dataframe of shifted periods

In [21]:
pd.concat([df.consumption.shift(i).rename(f't_{-i}') for i in range(5)], axis=1).head(10)

,t_0,t_-1,t_-2,t_-3,t_-4
0,25461,NaN,NaN,NaN,NaN
1,28972,25461.0,NaN,NaN,NaN
2,30814,28972.0,25461.0,NaN,NaN
3,29799,30814.0,28972.0,25461.0,NaN
4,28900,29799.0,30814.0,28972.0,25461.0
5,28218,28900.0,29799.0,30814.0,28972.0
6,29732,28218.0,28900.0,29799.0,30814.0
7,28397,29732.0,28218.0,28900.0,29799.0
8,24886,28397.0,29732.0,28218.0,28900.0
9,37937,24886.0,28397.0,29732.0,28218.0


A nice special case comes when using a Datetime Index like we did before. Then we can use `.tshift` and get some nice benefits

In [22]:
df = df.set_index('date')

We don't get any missing values, as we are simply incrementing the index by one period

In [23]:
df.tshift(1).head()

,median_temp,min_temp,max_temp,precip,weekend,consumption,season,high_consumption,consumption_group
date,,,,,,,,,
2015-01-02,27.30,23.9,32.5,0.0,0,25461,winter,1,medium
2015-01-03,27.02,24.5,33.5,0.0,0,28972,winter,1,high
2015-01-04,24.82,22.4,29.9,0.0,1,30814,winter,1,high
2015-01-05,23.98,21.5,28.6,1.2,1,29799,winter,1,high
2015-01-06,23.82,21.0,28.3,0.0,0,28900,winter,1,high


We can set different frequencies to shift by - for example, using 'M' "rounds up" to the nearest month end

In [24]:
df.tshift(1, freq='M').head()

,median_temp,min_temp,max_temp,precip,weekend,consumption,season,high_consumption,consumption_group
date,,,,,,,,,
2015-01-31,27.30,23.9,32.5,0.0,0,25461,winter,1,medium
2015-01-31,27.02,24.5,33.5,0.0,0,28972,winter,1,high
2015-01-31,24.82,22.4,29.9,0.0,1,30814,winter,1,high
2015-01-31,23.98,21.5,28.6,1.2,1,29799,winter,1,high
2015-01-31,23.82,21.0,28.3,0.0,0,28900,winter,1,high


# Stacking & Unstacking & MultiIndex
Stacking and unstacking are nice tools to manipulate our data into the form that we want. We often use it when we end up with a `MultiIndex` - an index with multiple `levels`. The most frequent way to end up with a `MultiIndex` is to do a groupby on multiple columns.

In [40]:
# Is there a difference in weekend consumption between seasons?
weekend_season = df.groupby(['season', 'consumption_group']).max_temp.mean()
weekend_season

season  consumption_group
autumn  low                  23.000000
        medium               26.482143
        high                 30.465789
spring  low                  24.097500
        medium               25.793548
        high                 27.100000
summer  low                  21.640000
        medium               25.348387
        high                 27.085714
winter  low                  27.188235
        medium               29.716129
        high                 31.019048
Name: max_temp, dtype: float64

This Series has two `levels` in its index - `season` and `consumption_group`. How can I access these values?

In [41]:
# This works
weekend_season.loc['autumn']

consumption_group
low       23.000000
medium    26.482143
high      30.465789
Name: max_temp, dtype: float64

In [42]:
# This doesn't
weekend_season.loc['low']

KeyError: 'low'

How would I get only the low consumption group?

I can use tuples to dig into a level and even do slices

In [52]:
# Get low consumption in autumn
weekend_season.loc[('autumn', 'low')]

22.999999999999996

In [55]:
# Get all values between autumn, low to spring, medium
weekend_season.loc[('autum', 'low'):('spring', 'medium')]

season  consumption_group
autumn  low                  23.000000
        medium               26.482143
        high                 30.465789
spring  low                  24.097500
        medium               25.793548
Name: max_temp, dtype: float64

What if I want all seasons, but only the low consumption group?

I could swap the levels in the index - this is a neat trick and can be useful sometimes

In [58]:
weekend_season.swaplevel()#.loc['low']

consumption_group  season
low                autumn    23.000000
medium             autumn    26.482143
high               autumn    30.465789
low                spring    24.097500
medium             spring    25.793548
high               spring    27.100000
low                summer    21.640000
medium             summer    25.348387
high               summer    27.085714
low                winter    27.188235
medium             winter    29.716129
high               winter    31.019048
Name: max_temp, dtype: float64

The best way is to use the IndexSlice convenience slicer - then we can just use our familiar `:` syntax to indicate "everything from this level" and it works on as many levels as you might have!

In [60]:
# The recommended method!
weekend_season.loc[pd.IndexSlice[:, 'low']]

season
autumn    23.000000
spring    24.097500
summer    21.640000
winter    27.188235
Name: max_temp, dtype: float64

In [63]:
df.groupby(['season', 'consumption_group', 'weekend']).max_temp.mean()#.loc[pd.IndexSlice[:, 'low', :]]

season  consumption_group  weekend
autumn  low                0          24.076190
                           1          17.350000
        medium             0          27.776190
                           1          22.600000
        high               0          31.873913
                           1          28.306667
spring  low                0          24.097500
        medium             0          26.809524
                           1          23.660000
        high               0          29.900000
                           1          26.441176
summer  low                0          21.926316
                           1          16.200000
        medium             0          26.245833
                           1          22.271429
        high               0          28.175000
                           1          26.829412
winter  low                0          27.025000
                           1          29.800000
        medium             0          30.057143
     

Another way to manipulate `MultiIndex` are using `.stack` and `.unstack`. Using these, you can "pivot" levels from an index into columns and viceversa - it's a common idiom to do a `.unstack` after a groupby!

In [66]:
# consumption_group is now our columns
weekend_season.unstack()

consumption_group,low,medium,high
season,,,
autumn,23.000000,26.482143,30.465789
spring,24.097500,25.793548,27.100000
summer,21.640000,25.348387,27.085714
winter,27.188235,29.716129,31.019048


In [70]:
# The outermost level is now stacked
weekend_season.unstack(level=0)

season,autumn,spring,summer,winter
consumption_group,,,,
low,23.000000,24.097500,21.640000,27.188235
medium,26.482143,25.793548,25.348387,29.716129
high,30.465789,27.100000,27.085714,31.019048


Stacking is not as common, as ending up with a `MultiIndex` on columns is not as common, but it does what one would expect - pivot from columns to index

In [77]:
agg_groupby = df.groupby('season').agg({"consumption": ['mean', 'sum', 'std'],
                                        "max_temp": ['mean', 'sum', 'std']
                                       })
agg_groupby

consumption                         max_temp                  
                mean      sum          std       mean     sum       std
season                                                                 
autumn  26242.186813  2388039  4530.842639  27.189011  2474.2  4.817156
spring  24147.836957  2221601  3844.770409  25.354348  2332.6  3.079547
summer  24040.750000  2211749  4394.841078  24.132609  2220.2  3.752716
winter  27223.444444  2450110  3988.602594  29.846667  2686.2  3.124359

In [79]:
# The aggregations are now in the index
agg_groupby.stack()

consumption     max_temp
season                                
autumn mean  2.624219e+04    27.189011
       sum   2.388039e+06  2474.200000
       std   4.530843e+03     4.817156
spring mean  2.414784e+04    25.354348
       sum   2.221601e+06  2332.600000
       std   3.844770e+03     3.079547
summer mean  2.404075e+04    24.132609
       sum   2.211749e+06  2220.200000
       std   4.394841e+03     3.752716
winter mean  2.722344e+04    29.846667
       sum   2.450110e+06  2686.200000
       std   3.988603e+03     3.124359

In [81]:
# Pass the level parameter to stack the other level
agg_groupby.stack(level=0)

mean          std        sum
season                                                  
autumn consumption  26242.186813  4530.842639  2388039.0
       max_temp        27.189011     4.817156     2474.2
spring consumption  24147.836957  3844.770409  2221601.0
       max_temp        25.354348     3.079547     2332.6
summer consumption  24040.750000  4394.841078  2211749.0
       max_temp        24.132609     3.752716     2220.2
winter consumption  27223.444444  3988.602594  2450110.0
       max_temp        29.846667     3.124359     2686.2